<center><h1>Cross Validation and Hyperparameters</h1></center>
<center><h3>Paul Stey</h3></center>

# What is Cross Validation?
- we want to find the best hyper-parameters of our ML algorithms
   - fit model to training data (`.fit(X_train,y_train)`)
   - evaluate model on CV set (`.predict(X_CV,y_CV)`)
   - we find hyper-parameter values that optimize the CV score
- we want to know how the model will perform on previously unseen data
   - apply our final model on the test set (`.predict(X_test,y_test)`)
   
- we need to split the data into parts!

## How should we split the data into train/CV/test?

- data is **Independent and Identically Distributed** (iid)
   - all samples stem from the same generative process and that the generative process is assumed to have no memory of past generated samples
   - identify cats and dogs on images
   - predict if someone's salary is above or below 50k
- examples of not iid data:
   - data generated by time-dependent processes
   - data has group structure (samples collected from e.g., different subjects, experiments, measurement devices)

## CV steps of iid data to avoid mistakes 
- shuffle and split the data
- preprocess (fit_transform train, transform the rest)
- decide on the evaluation metric
- decide ML algo, which hyper-parameters you tune, and what values you want to try
- loop over all combinations and save train and CV scores
- find best model based on optimal CV score
- report test score using the best model
- repeat a couple of times with different random states to estimate uncertainty

## Splitting strategies for iid data: basic approach
- the basic aproach:
   - 60% train, 20% CV, 20% test 
   - the ratios can vary somewhat but the training set should contain most of your points
   - if you redo the split with a different random state, the results will change
      - repeat the split a couple of times to measure model uncertainty due to splitting

### Let's put everything together!

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer


np.random.seed(10)


def true_fun(X):
    return 2*X + np.cos(4 * np.pi * X)

n_samples = 1000

X = np.random.randn(n_samples)
y = true_fun(X) + np.random.randn(n_samples) * 0.3

In [ ]:
plt.scatter(X, y, marker = '.')
plt.rcParams['figure.figsize'] = (10, 7)

In [ ]:
def ml_pipeline_basic(X, y, random_state):
    # split the data
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)
    X_train, X_CV, y_train, y_CV = train_test_split(X_other, y_other, test_size=0.25, random_state = random_state)
    
    # simple preprocessing
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_CV = scaler.transform(X_CV)
    X_test = scaler.transform(X_test)
    
    # tune ridge hyper-parameter, alpha
    alpha = np.logspace(-3, 4, num = 8)
    train_score = []
    CV_score = []
    regs = []
    
    for a in alpha:
        reg = Ridge(alpha = a)
        reg.fit(X_train,y_train)
        train_score.append(mean_squared_error(y_train, reg.predict(X_train)))
        CV_score.append(mean_squared_error(y_CV, reg.predict(X_CV)))
        regs.append(reg)
    
    # find the best alpha
    idx_best = np.argmin(CV_score)
    best_alpha = alpha[idx_best]
    
    # grab the best model
    reg = regs[idx_best]
    
    # calculate holdout score
    test_score = mean_squared_error(y_test,reg.predict(X_test))

    return best_alpha, np.min(CV_score), test_score

In [ ]:

CV_scores = []
test_scores = []
for i in range(10):
    best_alpha, CV_score, test_score = ml_pipeline_basic(X[:, np.newaxis], y, i*42 )
    CV_scores.append(CV_score)
    test_scores.append(test_score)


print('CV MSE:', np.around(np.mean(CV_scores),2),'+/-',np.around(np.std(CV_scores),2))
print('test MSE:', np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

## Splitting strategies for iid data, k-fold cross validation

<center><img src="images/grid_search_cross_validation.png" width="600"></center>


In [ ]:
def ml_pipeline_kfold(X,y,random_state,n_folds):
    # split the data
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state)
    CV_scores = []
    test_scores = []
    
    # k folds - each fold will give us a CV and a test score
    kf = KFold(n_splits=n_folds, shuffle = True,random_state=random_state)
    
    for train_index, CV_index in kf.split(X_other,y_other):
        X_train, X_CV = X_other[train_index], X_other[CV_index]
        y_train, y_CV = y_other[train_index], y_other[CV_index]
        
        # simple preprocessing
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_c = scaler.transform(X_CV)
        X_t = scaler.transform(X_test)
        
        # tune ridge hyper-parameter, alpha
        alpha = np.logspace(-3, 4, num=8)
        train_score = []
        CV_score = []
        regs = []
        
        for a in alpha:
            reg = Ridge(alpha = a)
            reg.fit(X_train,y_train)
            train_score.append(mean_squared_error(y_train,reg.predict(X_train)))
            CV_score.append(mean_squared_error(y_CV,reg.predict(X_c)))
            regs.append(reg)
        
        # find the best alpha in this fold
        best_alpha = alpha[np.argmin(CV_score)]
        
        # grab the best model
        reg = regs[np.argmin(CV_score)]
        CV_scores.append(np.min(CV_score))
        
        # calculate test score using thee best model
        test_scores.append(mean_squared_error(y_test,reg.predict(X_t)))
    
    return CV_scores,test_scores

In [ ]:

CV_scores, test_scores = ml_pipeline_kfold(X[:,np.newaxis],y,42,5)

print('CV MSE:',np.around(np.mean(CV_scores),2),'+/-',np.around(np.std(CV_scores),2))
print('test MSE:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),))

## Some considerations
- 1) lots of lines of code were written, mistakes can be easily made!
- 2) kfold CV uses the same test set, so we do not estimate the uncertainty from random test sets
   - test score uncertainty is lower than in the basic approach
- 3) both approaches (basic and kfold) can fail if the data is imbalanced
   - if one class is infrequent, it can happen that one set or one fold contains 0 points from the rare class
   - sklearn will raise an error in that case
- 4) neither of these approaches work, if data is not iid!

# Using `GridSearchCV` and pipeline in k-fold CV

In [ ]:
def ml_pipeline_kfold(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)
    
    # splitter for _other
    kf = KFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    
    # create the pipeline: preprocessor + supervised ML method
    scaler = StandardScaler()
    pipe = make_pipeline(scaler,Ridge())
    
    # the parameter(s) we want to tune
    param_grid = {'ridge__alpha': np.logspace(-3,4,num=8)}
    
    # prepare gridsearch
    grid = GridSearchCV(pipe, 
                        param_grid = param_grid,
                        scoring = make_scorer(mean_squared_error, greater_is_better=False),
                        cv = kf, 
                        return_train_score = True)
    
    # do kfold CV on _other
    grid.fit(X_other, y_other)
    
    return grid, grid.score(X_test, y_test)

In [ ]:

grid, test_score = ml_pipeline_kfold(X[:,np.newaxis],y,42,5)
results = pd.DataFrame(grid.cv_results_)
print('CV MSE:',-np.around(results[results['rank_test_score'] == 1]['mean_test_score'].values[0],2),\
      '+/-',np.around(results[results['rank_test_score'] == 1]['std_test_score'].values[0],2))
print('test MSE:',-np.around(test_score,2))
results

